In [136]:
from selenium import webdriver
import time
import bs4
import pandas as pd
from bs4 import BeautifulSoup
import requests
driver = webdriver.Firefox(executable_path = 'C:\Program Files\Mozilla Firefox\geckodriver')
driver.get("http://scit.bjtu.edu.cn/cms/staff/?kind=0&cat=12&page=1")
soup = BeautifulSoup(driver.page_source,'lxml')

In [139]:
#找到列表中字符串开始为固定值的都是哪些
def choose_teacher(l,s):
    len_of_s = len(s)#计算字符串的长度
    start = 0
    for i in l:
        if(i[:len_of_s]==s):
            start = l.index(i)
            break
    end = len(l)
    for j in l[start:]:
        if(j[:len_of_s]!=s):
            end = l.index(j)
            break
    return(start,end)

In [145]:
teacher_all_message = []
for i in range(16):
    i = i+1
    driver = webdriver.Firefox(executable_path = 'C:\Program Files\Mozilla Firefox\geckodriver')
    driver.get("http://scit.bjtu.edu.cn/cms/staff/?kind=0&cat=12&page="+str(i))
    soup = BeautifulSoup(driver.page_source,'lxml')
    a_labels = soup.find_all('a',attrs = {'href':True})#找到链接的位置在哪
    l = []
    for i in a_labels:
        l.append(str(i))
    url_all = []
    for a in a_labels:
        url_all.append(a.get('href'))#把网页都选出来
    start_and_end = choose_teacher(url_all,'/cms/staff/')
    url_all = url_all[start_and_end[0]:start_and_end[1]]
    for teacher_url in url_all:
        page = requests.get('http://scit.bjtu.edu.cn'+teacher_url)
        page.encoding = 'utf-8'
        te = BeautifulSoup(page.text)
        name = te.find('div',{'class':'page-header'})#找出教授的名字
        message = te.find('div',{'class':'col-md-10'}).find('div',{'class':'col-md-10'}).find_all('p')#找教授的各个信息
        l = []
        l.append(name.text)
        for i in message:
            l.append(i.text)
        director = te.find('div',{'class':'alert alert-success'}).find_next_sibling().text#找出科研方向
        l.append(director)
        teacher_all_message.append(l)

In [149]:
pd.DataFrame(teacher_all_message).to_excel('C:\\Users\\NYDSJ\\Desktop\\北交大老师.xls')

In [133]:
teacher_all_message = []
for teacher_url in url_all:
    page = requests.get('http://scit.bjtu.edu.cn'+teacher_url)
    page.encoding = 'utf-8'
    te = BeautifulSoup(page.text)
    name = te.find('div',{'class':'page-header'})#找出教授的名字
    message = te.find('div',{'class':'col-md-10'}).find('div',{'class':'col-md-10'}).find_all('p')#找教授的各个信息
    l = []
    l.append(name.text)
    for i in message:
        l.append(i.text)
    director = te.find('div',{'class':'alert alert-success'}).find_next_sibling().text#找出科研方向
    l.append(director)
    teacher_all_message.append(l)

In [147]:
len(teacher_all_message)

189